In [2]:
import collections
import functools
import IPython
import itertools
import numpy as np
import os
import pandas as pd
import pathlib
import pypandoc
import requests
import string
import sys
import time
import typing
import us
import zipfile

from datetime import datetime
from tqdm.notebook import tqdm_notebook

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_pipeline.utils import remove_all_from_dir, get_excel_column_name
from data_pipeline.etl.base import ExtractTransformLoad
from data_pipeline.etl.sources.census.etl_utils import get_state_information
from data_pipeline.etl.sources.ejscreen_areas_of_concern.etl import (
    EJSCREENAreasOfConcernETL,
)

from data_pipeline.score import field_names

# Suppress scientific notation in pandas (this shows up for census tract IDs)
pd.options.display.float_format = "{:.2f}".format

%load_ext lab_black

In [20]:
# Set some global parameters
DATA_DIR = pathlib.Path.cwd().parent / "data"
TEMP_DATA_DIR = DATA_DIR / "tmp"

time_str = time.strftime("%Y%m%d-%H%M%S")

COMPARISON_OUTPUTS_DIR = DATA_DIR / "comparison_outputs" / time_str

# Make the dirs if they don't exist
TEMP_DATA_DIR.mkdir(parents=True, exist_ok=True)
COMPARISON_OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

CEJST_PRIORITY_COMMUNITY_THRESHOLD = 0.75

# Name fields using variables. (This makes it easy to reference the same fields frequently without using strings
# and introducing the risk of misspelling the field name.)
GEOID_STATE_FIELD_NAME = "GEOID10_STATE"
COUNTRY_FIELD_NAME = "Country"

# Define some suffixes
POPULATION_SUFFIX = " (priority population)"

CALENVIROSCREEN_SCORE_FIELD = "calenviroscreen_score"
CALENVIROSCREEN_PERCENTILE_FIELD = "calenviroscreen_percentile"
CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD = "calenviroscreen_priority_community"

# Since "Persistent Poverty Census Tract" is labeled in both the score file (at the CBG level) and this tract file,
# rename this field so it's easy to access the tract-level scores directly.

PERSISTENT_POVERTY_TRACT_LEVEL_FIELD = "Persistent Poverty, Tract Level"
PERSISTENT_POVERTY_CBG_LEVEL_FIELD = "Persistent Poverty Census Tract"

In [38]:
def _read_tract_level_data(
    data_path,
    tract_field_name=ExtractTransformLoad.GEOID_TRACT_FIELD_NAME,
    add_state=False,
    state_field=None,
    priority_community_field=None,
    rename_dict=None,
):
    loaded_data = pd.read_csv(
        data_path,
        dtype={tract_field_name: str, priority_community_field: bool},
    )
    assert (loaded_data[tract_field_name].str.len().min() == 11) & (
        loaded_data[tract_field_name].str.len().max() == 11
    ), "Something is goofy with census tracts here"
    if add_state:
        loaded_data.loc[:, state_field] = loaded_data.loc[:, tract_field_name].str[0:2]
    return loaded_data.set_index(tract_field_name)

In [52]:
import pathlib

str(pathlib.PosixPath("~/j40"))

'~/j40'

In [49]:
cejst_df = _read_tract_level_data(
    **{"add_state": True, "state_field": GEOID_STATE_FIELD_NAME},
)

calenviroscreen_df = _read_tract_level_data(
    **{
        "data_path": DATA_DIR / "dataset" / "calenviroscreen4" / "data06.csv",
        "priority_community_field": CALENVIROSCREEN_PRIORITY_COMMUNITY_FIELD,
    }
)
mapping_for_ej_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "mapping_for_ej" / "co_va.csv",
    priority_community_field=field_names.MAPPING_FOR_EJ_PRIORITY_COMMUNITY_FIELD,
)

persistent_poverty_df = _read_tract_level_data(
    **{
        "data_path": DATA_DIR / "dataset" / "persistent_poverty" / "usa.csv",
        "priority_community_field": PERSISTENT_POVERTY_CBG_LEVEL_FIELD,
        "rename_dict": {
            PERSISTENT_POVERTY_CBG_LEVEL_FIELD: PERSISTENT_POVERTY_TRACT_LEVEL_FIELD
        },
    }
)

mapping_inequality_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "mapping_inequality" / "usa.csv"
)
cdc_svi_df = _read_tract_level_data(DATA_DIR / "dataset" / "cdc_svi_index" / "usa.csv")
maryland_ejscreen_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "maryland_ejscreen" / "maryland.csv"
)
energy_definition_alternative_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "energy_definition_alternative_draft" / "usa.csv"
)
michigan_ejscreen_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "michigan_ejscreen" / "michigan_ejscreen.csv"
)
epa_rsei_df = _read_tract_level_data(
    data_path=DATA_DIR / "dataset" / "epa_rsei" / "usa.csv"
)

/var/folders/z6/9czv8cpx2hvcycd6slvfp4ph0000gq/T/ipykernel_25820/4211726722.py:1: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  cejst_df = _read_tract_level_data(


So I think we should do LESS here

One of two things should happen:
_option A_
(1) All "background" indices -- demographics and CEJST stuff
(2) _A SINGLE_ comparison to another dataframe

_option B_ 
(1) All "background" indices -- demographics and CEJST stuff
(2) _A SINGLE_ comparison to an old score

In [50]:
joined_data = pd.concat(
    [
        cejst_df,
        calenviroscreen_df,
        mapping_for_ej_df,
        persistent_poverty_df,
        mapping_inequality_df,
        cdc_svi_df,
        maryland_ejscreen_df,
        energy_definition_alternative_df,
        michigan_ejscreen_df,
        epa_rsei_df,
    ],
    axis=1,
)

In [10]:
# Load mapping inequality data
HOLC_FACTORS = [
    field_names.HOLC_GRADE_D_TRACT_20_PERCENT_FIELD,
    field_names.HOLC_GRADE_D_TRACT_50_PERCENT_FIELD,
    field_names.HOLC_GRADE_D_TRACT_75_PERCENT_FIELD,
]

mapping_inequality_df = pd.read_csv(
    mapping_inequality_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)

mapping_inequality_df

,GEOID10_TRACT,Percent of tract that is HOLC Grade D,Tract is >20% HOLC Grade D,Tract is >50% HOLC Grade D,Tract is >75% HOLC Grade D
0,01073000100,0.42,True,False,False
1,01073000300,0.93,True,True,True
2,01073000400,0.36,True,False,False
3,01073000500,0.65,True,True,False
4,01073000700,0.41,True,False,False
...,...,...,...,...,...
7273,55139001100,0.29,True,False,False
7274,55139001200,0.42,True,False,False
7275,55139001300,0.16,False,False,False
7276,55139001400,0.04,False,False,False


In [11]:

cdc_svi_index_df = pd.read_csv(
    cdc_svi_index_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)
cdc_svi_index_df.head()

,GEOID10_TRACT,SVI - Socioeconomic Index (percentile),SVI - Household Composition Index (percentile),SVI- Minority Status/Language Index (percentile),SVI- Housing Type/Transportation Index (percentile),Overall rank for Social Vulnerability Indices (percentile),At or above 90 for overall percentile ranking according to Social Vulnerability Indices
0,01015981901,NaN,NaN,NaN,NaN,NaN,False
1,01015981902,NaN,NaN,NaN,NaN,NaN,False
2,01015981903,NaN,NaN,NaN,NaN,NaN,False
3,01097003605,NaN,0.00,0.27,NaN,NaN,False
4,01097990000,NaN,NaN,NaN,NaN,NaN,False


In [12]:
# Load Maryland EJScreen

maryland_ejscreen_df = pd.read_csv(
    maryland_ejscreen_data_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)

maryland_ejscreen_df.tail()

,GEOID10_TRACT,Maryland Environmental Justice Score,Maryland EJSCREEN Priority Community
1379,24027601107,0.66,True
1380,24027602201,0.30,False
1381,24027605504,0.28,False
1382,24027606904,0.43,False
1383,24027606906,0.54,False


In [13]:
# Load alternative energy-related definition


energy_definition_alternative_draft_df = pd.read_csv(
    energy_definition_alternative_draft_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)

energy_definition_alternative_draft_df

,GEOID10_TRACT,Energy-related alternative definition of communities,Coal employment,Outage Events,Homelessness,Disabled population,Outage Duration,Job Access,Fossil energy employment,Food Desert,Incomplete Plumbing,Non-grid-connected heating fuel,Parks,Greater than 30 min commute,Internet Access,Mobile Home,Single Parent,Transportation Costs
0,35029000500,True,0.00,0.00,0.01,0.22,0.00,-1.40,0.02,1.00,0.00,0.36,0.00,0.25,30.70,0.53,0.75,45.00
1,48311950100,False,0.00,0.00,0.01,0.17,0.00,-1.30,0.01,1.00,0.02,0.04,-1.00,0.31,21.90,0.33,0.18,34.00
2,48131950500,True,0.00,0.00,0.01,0.29,0.00,-1.50,0.01,1.00,0.02,0.15,0.00,0.54,33.90,0.15,0.45,40.00
3,48247950200,False,0.00,1.00,0.01,0.22,1140.00,-3.00,0.01,0.00,0.00,0.01,0.00,0.33,29.40,0.11,0.53,37.00
4,48247950400,True,0.00,1.00,0.01,0.22,1140.00,-2.70,0.01,1.00,0.01,0.03,0.00,0.36,53.10,0.11,0.78,41.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73051,55111000500,False,NaN,NaN,NaN,0.12,NaN,NaN,NaN,NaN,0.00,0.30,NaN,NaN,NaN,0.10,0.32,NaN
73052,55111000700,False,NaN,NaN,NaN,0.11,NaN,NaN,NaN,NaN,0.01,0.07,NaN,NaN,NaN,0.00,0.25,NaN
73053,55111000800,False,NaN,NaN,NaN,0.14,NaN,NaN,NaN,NaN,0.01,0.46,NaN,NaN,NaN,0.05,0.22,NaN
73054,55111000900,False,NaN,NaN,NaN,0.11,NaN,NaN,NaN,NaN,0.03,0.78,NaN,NaN,NaN,0.02,0.25,NaN


In [14]:
# Load Michigan EJSCREEN

michigan_ejscreen_df = pd.read_csv(
    michigan_ejscreen_data_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)

michigan_ejscreen_df.head()

,GEOID10_TRACT,Michigan EJSCREEN Score Field,Michigan EJSCREEN Percentile Field,Michigan EJSCREEN Priority Community
0,26081003900,93.99,1.00,True
1,26077000202,91.95,1.00,True
2,26077000300,90.17,1.00,True
3,26081003800,89.50,1.00,True
4,26081004000,87.74,1.00,True


In [15]:
# Load EPA RSEI EJSCREEN

epa_rsei_df = pd.read_csv(
    epa_rsei_data_path,
    dtype={ExtractTransformLoad.GEOID_TRACT_FIELD_NAME: "string"},
)

epa_rsei_df.head()

,GEOID10_TRACT,Number of facilities affecting the tract,Number of releases affecting the tract,Number of chemicals affecting the tract,Average toxicity-weighted concentration of the cells in the tract,RSEI Risk Score,RSEI Risk Score (Cancer toxicity weights),RSEI Risk Score (Noncancer toxicity weights),Sum of the population of the cells in the tract,At or above 75 for overall percentile for the RSEI score,RSEI Risk Score (percentile)
0,36047023200,193,904,96,485.18,759.71,610.32,154.85,6199.22,False,0.53
1,36029007202,84,376,81,227.37,79.50,63.05,14.81,1601.87,False,0.21
2,42003429201,144,1624,251,2705.16,2553.95,2447.72,113.75,4527.60,False,0.74
3,17201004100,80,365,57,3626.62,3984.97,3969.13,27.06,5766.72,True,0.82
4,47065011203,79,358,79,1133.40,1885.51,1410.31,447.77,7820.66,False,0.69


In [16]:
# Join all dataframes that use tracts
census_tract_dfs = [
    cejst_df,
    calenviroscreen_df,
    persistent_poverty_df,
    mapping_inequality_df,
    epa_rsei_df,
    maryland_ejscreen_df,
    energy_definition_alternative_draft_df,
    michigan_ejscreen_df,
    cdc_svi_index_df,
    mapping_for_ej_df,
]

merged_df = functools.reduce(
    lambda left, right: pd.merge(
        left=left,
        right=right,
        on=ExtractTransformLoad.GEOID_TRACT_FIELD_NAME,
        how="outer",
    ),
    census_tract_dfs,
)

tract_values = merged_df[ExtractTransformLoad.GEOID_TRACT_FIELD_NAME].str.len().unique()
if any(tract_values != [11]):
    print(tract_values)
    raise ValueError("Some of the census tract data has the wrong length.")

if len(merged_df) > ExtractTransformLoad.EXPECTED_MAX_CENSUS_TRACTS:
    raise ValueError(f"Too many rows in the join: {len(merged_df)}.")

merged_df.head()

,GEOID10_TRACT,Persistent Poverty Census Tract,Housing burden (percent),Total population,Median household income (% of state median household income),Current asthma among adults aged greater than or equal to 18 years,Coronary heart disease among adults aged greater than or equal to 18 years,Cancer (excluding skin cancer) among adults aged greater than or equal to 18 years,Current lack of health insurance among adults aged 18-64 years,Diagnosed diabetes among adults aged greater than or equal to 18 years,...,Michigan EJSCREEN Priority Community,SVI - Socioeconomic Index (percentile),SVI - Household Composition Index (percentile),SVI- Minority Status/Language Index (percentile),SVI- Housing Type/Transportation Index (percentile),Overall rank for Social Vulnerability Indices (percentile),At or above 90 for overall percentile ranking according to Social Vulnerability Indices,Mapping for Environmental Justice Final Percentile,Mapping for Environmental Justice Final Score,Mapping for Environmental Justice Priority Community
0,01073001100,True,0.28,4897.00,0.73,11.20,7.20,6.70,16.60,19.30,...,NaN,0.62,0.98,0.50,0.40,0.69,False,NaN,NaN,NaN
1,01073001400,True,0.18,1906.00,0.71,11.10,9.10,7.30,21.40,22.40,...,NaN,0.80,0.57,0.85,0.23,0.68,False,NaN,NaN,NaN
2,01073002000,False,0.44,4215.00,0.54,13.50,9.50,6.10,25.40,22.80,...,NaN,0.95,0.82,0.73,0.96,0.97,True,NaN,NaN,NaN
3,01073003802,False,0.41,5149.00,0.77,12.00,6.60,5.60,20.90,18.60,...,NaN,0.85,0.81,0.50,0.49,0.76,False,NaN,NaN,NaN
4,01073004000,True,0.47,2621.00,0.37,13.10,10.00,6.30,24.50,25.00,...,NaN,0.96,0.71,0.86,0.85,0.95,True,NaN,NaN,NaN


In [17]:
# Special handling for HOLC.
# Fill in the null HOLC values as `False`. Otherwise the comparison tool will not run comparisons in states
# without HOLC scores, and for HOLC, we'd like to see it across the whole US.
for holc_factor in HOLC_FACTORS:
    merged_df[holc_factor] = merged_df[holc_factor].fillna(False)

merged_df[HOLC_FACTORS].head()

,Tract is >20% HOLC Grade D,Tract is >50% HOLC Grade D,Tract is >75% HOLC Grade D
0,True,False,False
1,True,True,True
2,False,False,False
3,True,False,False
4,True,False,False


In [18]:
# Define a namedtuple for indices.
Index = collections.namedtuple(
    typename="Index",
    field_names=[
        "method_name",
        "priority_communities_field",
    ],
)

# Define the indices used for CEJST scoring (`census_block_group_indices`) as well as comparison
# (`census_tract_indices`).
definition_l_factors = [
    field_names.L_CLIMATE,
    field_names.L_ENERGY,
    field_names.L_TRANSPORTATION,
    field_names.L_HOUSING,
    field_names.L_POLLUTION,
    field_names.L_WATER,
    field_names.L_HEALTH,
    field_names.L_WORKFORCE,
    # Also include a combined factor for all the non-workforce elements.
    field_names.L_NON_WORKFORCE,
]

definition_m_factors = [
    field_names.M_CLIMATE,
    field_names.M_ENERGY,
    field_names.M_TRANSPORTATION,
    field_names.M_HOUSING,
    field_names.M_POLLUTION,
    field_names.M_WATER,
    field_names.M_HEALTH,
    field_names.M_WORKFORCE,
    # Also include a combined factor for all the non-workforce elements.
    field_names.M_NON_WORKFORCE,
]

census_tract_indices = (
    [
        Index(
            method_name="Definition M",
            priority_communities_field=field_names.SCORE_M_COMMUNITIES,
        ),
    ]
    + [
        Index(
            method_name="Definition L",
            priority_communities_field=field_names.SCORE_L_COMMUNITIES,
        ),
    ]
    # Insert indices for each of the factors from Definition M.
    # Note: since these involve no renaming, we write them using list comprehension.
    + [
        Index(
            method_name=factor,
            priority_communities_field=factor,
        )
        for factor in definition_m_factors
    ]
    # Insert indices for each of the factors from Definition L.
    # Note: since these involve no renaming, we write them using list comprehension.
    + [
        Index(
            method_name=factor,
            priority_communities_field=factor,
        )
        for factor in definition_l_factors
    ]
    + [
        Index(
            # Note: we're renaming Score G as NMTC Modified for clarity, since that's what Score G is under the hood.
            method_name="NMTC Modified",
            priority_communities_field=field_names.SCORE_G_COMMUNITIES,
        ),
        Index(
            method_name="NMTC",
            priority_communities_field="NMTC (communities)",
        ),
        Index(
            method_name="Score C",
            priority_communities_field="Score C (top 25th percentile)",
        ),
        Index(
            method_name="Score D (30th percentile)",
            priority_communities_field="Score D (top 30th percentile)",
        ),
        Index(
            method_name="Score D (25th percentile)",
            priority_communities_field="Score D (top 25th percentile)",
        ),
        Index(
            method_name="Score F",
            priority_communities_field=field_names.SCORE_F_COMMUNITIES,
        ),
        Index(
            method_name="CalEnviroScreen 4.0",
            priority_communities_field="calenviroscreen_priority_community",
        ),
        Index(
            method_name="Mapping for EJ",
            priority_communities_field=field_names.MAPPING_FOR_EJ_PRIORITY_COMMUNITY_FIELD,
        ),
        Index(
            method_name="EPA RSEI Aggregate Microdata",
            priority_communities_field=field_names.EPA_RSEI_SCORE_THRESHOLD_FIELD,
        ),
        Index(
            method_name="Persistent Poverty",
            priority_communities_field=PERSISTENT_POVERTY_TRACT_LEVEL_FIELD,
        ),
        Index(
            method_name="Maryland EJSCREEN",
            priority_communities_field=field_names.MARYLAND_EJSCREEN_BURDENED_THRESHOLD_FIELD,
        ),
        Index(
            method_name=field_names.ENERGY_RELATED_COMMUNITIES_DEFINITION_ALTERNATIVE,
            priority_communities_field=field_names.ENERGY_RELATED_COMMUNITIES_DEFINITION_ALTERNATIVE,
        ),
        Index(
            method_name="CDC SVI Index",
            priority_communities_field=field_names.CDC_SVI_INDEX_THEMES_PRIORITY_COMMUNITY,
        ),
        Index(
            method_name="Michigan EJSCREEN",
            priority_communities_field=field_names.MICHIGAN_EJSCREEN_PRIORITY_COMMUNITY_FIELD,
        ),
    ]
    # Insert indices for each of the HOLC factors.
    # Note: since these involve no renaming, we write them using list comprehension.
    + [
        Index(
            method_name=factor,
            priority_communities_field=factor,
        )
        for factor in HOLC_FACTORS
    ]
)


ejscreen_areas_of_concern_census_block_group_indices = [
    Index(
        method_name="EJSCREEN Areas of Concern, National, 80th percentile",
        priority_communities_field=field_names.EJSCREEN_AREAS_OF_CONCERN_NATIONAL_80TH_PERCENTILE_COMMUNITIES_FIELD,
    ),
    Index(
        method_name="EJSCREEN Areas of Concern, National, 90th percentile",
        priority_communities_field=field_names.EJSCREEN_AREAS_OF_CONCERN_NATIONAL_90TH_PERCENTILE_COMMUNITIES_FIELD,
    ),
    Index(
        method_name="EJSCREEN Areas of Concern, National, 95th percentile",
        priority_communities_field=field_names.EJSCREEN_AREAS_OF_CONCERN_NATIONAL_95TH_PERCENTILE_COMMUNITIES_FIELD,
    ),
]

# Before including EJSCREEN AoC indicators are included, check whether or not the EJSCREEN AoC data is available locally.
if EJSCREENAreasOfConcernETL.ejscreen_areas_of_concern_data_exists():
    # Add EJSCREEN AoCs to all of the CBG indices.
    # TODO: When we get AoC data at the tract level, fix this.
    # Right now commenting this out to avoid merging CBG-level areas of concern on a tract-level CEJST definition.
    # census_block_group_indices.extend(
    #     ejscreen_areas_of_concern_census_block_group_indices
    # )
    pass
else:
    pass

# These fields will be used for statistical comparisons.
comparison_fields = [
    field_names.POVERTY_LESS_THAN_100_FPL_FIELD,
    field_names.POVERTY_LESS_THAN_200_FPL_FIELD,
    field_names.COLLEGE_ATTENDANCE_FIELD,
    field_names.MEDIAN_INCOME_AS_PERCENT_OF_AMI_FIELD,
    field_names.LINGUISTIC_ISO_FIELD,
    field_names.UNEMPLOYMENT_FIELD,
    field_names.HIGH_SCHOOL_ED_FIELD,
    field_names.MEDIAN_INCOME_FIELD,
    field_names.URBAN_HEURISTIC_FIELD,
    field_names.LIFE_EXPECTANCY_FIELD,
    field_names.HEALTH_INSURANCE_FIELD,
    field_names.PHYS_HEALTH_NOT_GOOD_FIELD,
    field_names.DIABETES_FIELD,
    field_names.LOW_READING_FIELD + field_names.PERCENTILE_FIELD_SUFFIX,
]

In [19]:
def get_state_distributions(
    df: pd.DataFrame, priority_communities_fields: typing.List[str]
) -> pd.DataFrame:
    """For each boolean field of priority communities, calculate distribution across states and territories."""

    # Ensure each field is boolean.
    for priority_communities_field in priority_communities_fields:
        if df[priority_communities_field].dtype != bool:
            print(f"Converting {priority_communities_field} to boolean.")

        # Calculate the population included as priority communities per tract. Will either be 0 or the population.
        df[f"{priority_communities_field}{POPULATION_SUFFIX}"] = (
            df[priority_communities_field]
            * df[field_names.COMBINED_CENSUS_TOTAL_POPULATION_2010]
        )

    def calculate_state_comparison(
        frame: pd.DataFrame, geography_field: str
    ) -> pd.DataFrame:
        """
        This method will be applied to a `group_by` object. Inherits some parameters from outer scope.

        """
        summary_dict = {}
        summary_dict[COUNTRY_FIELD_NAME] = frame[COUNTRY_FIELD_NAME].unique()[0]

        summary_dict["Analysis grouped by"] = geography_field

        if geography_field == COUNTRY_FIELD_NAME:
            summary_dict[GEOID_STATE_FIELD_NAME] = "00"
            summary_dict["Geography name"] = "(Entire USA)"

        if geography_field == GEOID_STATE_FIELD_NAME:
            state_id = frame[GEOID_STATE_FIELD_NAME].unique()[0]
            summary_dict[GEOID_STATE_FIELD_NAME] = state_id
            summary_dict["Geography name"] = us.states.lookup(state_id).name

            # Also add region information
            region_id = frame["region"].unique()[0]
            summary_dict["region"] = region_id

        if geography_field == "region":
            region_id = frame["region"].unique()[0]
            summary_dict["region"] = region_id
            summary_dict["Geography name"] = region_id

        if geography_field == "division":
            division_id = frame["division"].unique()[0]
            summary_dict["division"] = division_id
            summary_dict["Geography name"] = division_id

        total_tracts_in_geography = len(frame)
        total_population_in_geography = frame[
            field_names.COMBINED_CENSUS_TOTAL_POPULATION_2010
        ].sum()

        if geography_field == field_names.URBAN_HEURISTIC_FIELD:
            urban_flag = frame[field_names.URBAN_HEURISTIC_FIELD].unique()[0]
            summary_dict["Urban vs Rural"] = "Urban" if urban_flag else "Rural"
            summary_dict["Geography name"] = summary_dict["Urban vs Rural"]

        for priority_communities_field in priority_communities_fields:
            summary_dict[f"{priority_communities_field}{POPULATION_SUFFIX}"] = frame[
                f"{priority_communities_field}{POPULATION_SUFFIX}"
            ].sum()

            summary_dict[f"{priority_communities_field} (total tracts)"] = frame[
                f"{priority_communities_field}"
            ].sum()

            # Calculate some combinations of other variables.
            summary_dict[f"{priority_communities_field} (percent tracts)"] = (
                summary_dict[f"{priority_communities_field} (total tracts)"]
                / total_tracts_in_geography
            )

            summary_dict[f"{priority_communities_field} (percent population)"] = (
                summary_dict[f"{priority_communities_field}{POPULATION_SUFFIX}"]
                / total_population_in_geography
            )

            unwanted_keys = [
                f"{priority_communities_field}{POPULATION_SUFFIX}",
                f"{priority_communities_field} (total tracts)",
            ]

            # Remove unneeded columns:
            for unwanted_key in unwanted_keys:
                del summary_dict[unwanted_key]

        df = pd.DataFrame(summary_dict, index=[0])

        return df

    # Add a field for country so we can do aggregations across the entire country.
    df[COUNTRY_FIELD_NAME] = "USA"

    # First, run the comparison by the whole country
    usa_grouped_df = df.groupby(COUNTRY_FIELD_NAME)

    # Run the comparison function on the groups.
    usa_distribution_df = usa_grouped_df.progress_apply(
        lambda frame: calculate_state_comparison(
            frame, geography_field=COUNTRY_FIELD_NAME
        )
    )

    # Next, run the comparison by state
    state_grouped_df = df.groupby(GEOID_STATE_FIELD_NAME)

    # Run the comparison function on the groups.
    state_distribution_df = state_grouped_df.progress_apply(
        lambda frame: calculate_state_comparison(
            frame, geography_field=GEOID_STATE_FIELD_NAME
        )
    )

    # Next, run the comparison by region
    region_grouped_df = df.groupby("region")

    # Run the comparison function on the groups.
    region_distribution_df = region_grouped_df.progress_apply(
        lambda frame: calculate_state_comparison(frame, geography_field="region")
    )

    # Next, run the comparison by division
    division_grouped_df = df.groupby("division")

    # Run the comparison function on the groups.
    division_distribution_df = division_grouped_df.progress_apply(
        lambda frame: calculate_state_comparison(frame, geography_field="division")
    )

    # Next, run the comparison by urban/rural
    urban_grouped_df = df.groupby(field_names.URBAN_HEURISTIC_FIELD)

    # Run the comparison function on the groups.
    urban_grouped_df = urban_grouped_df.progress_apply(
        lambda frame: calculate_state_comparison(
            frame, geography_field=field_names.URBAN_HEURISTIC_FIELD
        )
    )

    # Combine the five
    combined_df = pd.concat(
        [
            usa_distribution_df,
            state_distribution_df,
            region_distribution_df,
            division_distribution_df,
            urban_grouped_df,
        ]
    )

    return combined_df


def write_state_distribution_excel(
    state_distribution_df: pd.DataFrame, file_path: pathlib.PosixPath
) -> None:
    """Write the dataframe to excel with special formatting."""
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_path, engine="xlsxwriter")

    # Convert the dataframe to an XlsxWriter Excel object. We also turn off the
    # index column at the left of the output dataframe.
    state_distribution_df.to_excel(writer, sheet_name="Sheet1", index=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    worksheet.autofilter(
        0, 0, state_distribution_df.shape[0], state_distribution_df.shape[1]
    )

    # Set a width parameter for all columns
    # Note: this is parameterized because every call to `set_column` requires setting the width.
    column_width = 15

    for column in state_distribution_df.columns:
        # Turn the column index into excel ranges (e.g., column #95 is "CR" and the range may be "CR2:CR53").
        column_index = state_distribution_df.columns.get_loc(column)
        column_character = get_excel_column_name(column_index)

        # Set all columns to larger width
        worksheet.set_column(f"{column_character}:{column_character}", column_width)

        # Special formatting for all percent columns
        # Note: we can't just search for `percent`, because that's included in the word `percentile`.
        if "percent " in column or "(percent)" in column:
            # Make these columns percentages.
            percentage_format = workbook.add_format({"num_format": "0%"})
            worksheet.set_column(
                f"{column_character}:{column_character}",
                column_width,
                percentage_format,
            )

        # Special formatting for columns that capture the percent of population considered priority.
        if "(percent population)" in column:
            column_ranges = (
                f"{column_character}2:{column_character}{len(state_distribution_df)+1}"
            )

            # Add green to red conditional formatting.
            worksheet.conditional_format(
                column_ranges,
                # Min: green, max: red.
                {
                    "type": "2_color_scale",
                    "min_color": "#00FF7F",
                    "max_color": "#C82538",
                },
            )

    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "bottom"}
    )

    # Overwrite both the value and the format of each header cell
    # This is because xlsxwriter / pandas has a known bug where it can't wrap text for a dataframe.
    # See https://stackoverflow.com/questions/42562977/xlsxwriter-text-wrap-not-working.
    for col_num, value in enumerate(state_distribution_df.columns.values):
        worksheet.write(0, col_num, value, header_format)

    writer.save()


fields_to_analyze = [index.priority_communities_field for index in census_tract_indices]

# Convert all indices to boolean
for field_to_analyze in fields_to_analyze:
    if "Areas of Concern" in field_to_analyze:
        print(f"Converting {field_to_analyze} to boolean.")

        merged_df[field_to_analyze] = merged_df[field_to_analyze].fillna(value=0)
        merged_df[field_to_analyze] = merged_df[field_to_analyze].astype(bool)


state_fips_codes = get_state_information(DATA_DIR)

merged_with_state_information_df = merged_df.merge(
    right=state_fips_codes, left_on=GEOID_STATE_FIELD_NAME, right_on="fips"
)

state_distribution_df = get_state_distributions(
    df=merged_with_state_information_df,
    priority_communities_fields=fields_to_analyze,
)

file_prefix = "Priority Tracts – Different geographic groupings"

state_distribution_df.to_csv(
    path_or_buf=COMPARISON_OUTPUTS_DIR / f"{file_prefix}.csv",
    na_rep="",
    index=False,
)

write_state_distribution_excel(
    state_distribution_df=state_distribution_df,
    file_path=COMPARISON_OUTPUTS_DIR / f"{file_prefix}.xlsx",
)

Converting calenviroscreen_priority_community to boolean.
Converting Mapping for Environmental Justice Priority Community to boolean.
Converting At or above 75 for overall percentile for the RSEI score to boolean.
Converting Persistent Poverty, Tract Level to boolean.
Converting Maryland EJSCREEN Priority Community to boolean.
Converting Energy-related alternative definition of communities to boolean.
Converting At or above 90 for overall percentile ranking according to Social Vulnerability Indices to boolean.
Converting Michigan EJSCREEN Priority Community to boolean.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
directory = COMPARISON_OUTPUTS_DIR / "tracts_basic_stats"
directory.mkdir(parents=True, exist_ok=True)

# TODO: this Excel-writing function is extremely similar to other Excel-writing functions in this notebook.
# Refactor to use the same Excel-writing function.
def write_basic_stats_excel(
    basic_stats_df: pd.DataFrame, file_path: pathlib.PosixPath
) -> None:
    """Write the dataframe to excel with special formatting."""
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_path, engine="xlsxwriter")

    # Convert the dataframe to an XlsxWriter Excel object. We also turn off the
    # index column at the left of the output dataframe.
    basic_stats_df.to_excel(writer, sheet_name="Sheet1", index=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    worksheet.autofilter(0, 0, basic_stats_df.shape[0], basic_stats_df.shape[1])

    # Set a width parameter for all columns
    # Note: this is parameterized because every call to `set_column` requires setting the width.
    column_width = 15

    for column in basic_stats_df.columns:
        # Turn the column index into excel ranges (e.g., column #95 is "CR" and the range may be "CR2:CR53").
        column_index = basic_stats_df.columns.get_loc(column)
        column_character = get_excel_column_name(column_index)

        # Set all columns to larger width
        worksheet.set_column(f"{column_character}:{column_character}", column_width)

        # Add green to red conditional formatting.
        column_ranges = f"{column_character}2:{column_character}{len(basic_stats_df)+1}"
        worksheet.conditional_format(
            column_ranges,
            # Min: green, max: red.
            {
                "type": "2_color_scale",
                "min_color": "#00FF7F",
                "max_color": "#C82538",
            },
        )

        # Special formatting for all percent columns
        # Note: we can't just search for `percent`, because that's included in the word `percentile`.
        if "percent " in column or "(percent)" in column or "Percent " in column:
            # Make these columns percentages.
            percentage_format = workbook.add_format({"num_format": "0%"})
            worksheet.set_column(
                f"{column_character}:{column_character}",
                column_width,
                percentage_format,
            )

    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "bottom"}
    )

    # Overwrite both the value and the format of each header cell
    # This is because xlsxwriter / pandas has a known bug where it can't wrap text for a dataframe.
    # See https://stackoverflow.com/questions/42562977/xlsxwriter-text-wrap-not-working.
    for col_num, value in enumerate(basic_stats_df.columns.values):
        worksheet.write(0, col_num, value, header_format)

    writer.save()


for index in census_tract_indices:
    print(f"Basic stats for {index.method_name}")
    temp_df = merged_df
    temp_df[index.priority_communities_field] = (
        temp_df[index.priority_communities_field] == True
    )

    grouped_df = temp_df.groupby(index.priority_communities_field).mean().reset_index()
    result_df = grouped_df[[index.priority_communities_field] + comparison_fields]
    result_df.to_csv(directory / f"{index.method_name} Basic Stats.csv", index=False)
    write_basic_stats_excel(
        basic_stats_df=result_df,
        file_path=directory / f"{index.method_name} Basic Stats.xlsx",
    )

Basic stats for Definition M
Basic stats for Definition L
Basic stats for Climate Factor (Definition M)
Basic stats for Energy Factor (Definition M)
Basic stats for Transportation Factor (Definition M)
Basic stats for Housing Factor (Definition M)
Basic stats for Pollution Factor (Definition M)
Basic stats for Water Factor (Definition M)
Basic stats for Health Factor (Definition M)
Basic stats for Workforce Factor (Definition M)
Basic stats for Any Non-Workforce Factor (Definition M)
Basic stats for Climate Factor (Definition L)
Basic stats for Energy Factor (Definition L)
Basic stats for Transportation Factor (Definition L)
Basic stats for Housing Factor (Definition L)
Basic stats for Pollution Factor (Definition L)
Basic stats for Water Factor (Definition L)
Basic stats for Health Factor (Definition L)
Basic stats for Workforce Factor (Definition L)
Basic stats for Any Non-Workforce Factor (Definition L)
Basic stats for NMTC Modified
Basic stats for NMTC
Basic stats for Score C
Basic

In [21]:
# Compare census tract scores to each other, running secondary analysis on
# characteristics of census tracts prioritized by one but not the other.
def get_census_tracts_score_comparison_df(
    df: pd.DataFrame,
    method_a_priority_census_tracts_field: str,
    method_b_priority_census_tracts_field: str,
    comparison_fields: typing.List[str],
) -> pd.DataFrame:
    """Compare tract scores to each other.

    This comparison method analyzes characteristics of those census tracts, based on whether or not they are prioritized
    or not by Method A and/or Method B.

    E.g., it might show that tracts prioritized by A but not B have a higher average income,
    or that tracts prioritized by B but not A have a lower percent of unemployed people.
    """
    fields_to_group_by = [
        method_a_priority_census_tracts_field,
        method_b_priority_census_tracts_field,
    ]

    df_subset = df[fields_to_group_by + comparison_fields]

    grouped_df = df_subset.groupby(
        fields_to_group_by,
        dropna=False,
    )

    # Take the mean of all fields.
    comparison_df = grouped_df.mean()

    # Also add in the count of census tracts.
    count_field_name = "Count of census tracts"
    comparison_df[count_field_name] = grouped_df.size().to_frame(count_field_name)

    comparison_df = comparison_df.reset_index()

    criteria_description_field_name = "Description of criteria"
    comparison_df[criteria_description_field_name] = comparison_df.apply(
        func=lambda row: f"Tracts that are {'not ' if row[method_a_priority_census_tracts_field] is False else ''}"
        + f"prioritized by {method_a_priority_census_tracts_field} "
        + f"and are {'not ' if row[method_b_priority_census_tracts_field] is False else ''}"
        + f"prioritized by {method_b_priority_census_tracts_field}",
        axis=1,
    )

    # Put criteria description column first.
    columns_to_put_first = (
        [criteria_description_field_name] + fields_to_group_by + [count_field_name]
    )
    new_column_order = columns_to_put_first + [
        col for col in comparison_df.columns if col not in columns_to_put_first
    ]

    comparison_df = comparison_df[new_column_order]

    # Rename fields to reflect the mean aggregation
    comparison_df.rename(
        mapper={
            comparison_field: f"{comparison_field} (mean of tracts)"
            for comparison_field in comparison_fields
        },
        axis=1,
        inplace=True,
    )

    return comparison_df


def write_census_tracts_score_comparison_excel(
    census_tracts_score_comparison_df: pd.DataFrame,
    file_path: pathlib.PosixPath,
) -> None:
    """Write the dataframe to excel with special formatting."""
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_path, engine="xlsxwriter")

    # Convert the dataframe to an XlsxWriter Excel object. We also turn off the
    # index column at the left of the output dataframe.
    census_tracts_score_comparison_df.to_excel(writer, sheet_name="Sheet1", index=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    worksheet.autofilter(
        0,
        0,
        census_tracts_score_comparison_df.shape[0],
        census_tracts_score_comparison_df.shape[1],
    )

    # Set a width parameter for all columns
    # Note: this is parameterized because every call to `set_column` requires setting the width.
    column_width = 15

    for column in census_tracts_score_comparison_df.columns:
        # Turn the column index into excel ranges (e.g., column #95 is "CR" and the range may be "CR2:CR53").
        column_index = census_tracts_score_comparison_df.columns.get_loc(column)
        column_character = get_excel_column_name(column_index)

        # Set all columns to larger width
        worksheet.set_column(f"{column_character}:{column_character}", column_width)

        # Add green to red conditional formatting.
        column_ranges = f"{column_character}2:{column_character}{len(census_tracts_score_comparison_df)+1}"
        worksheet.conditional_format(
            column_ranges,
            # Min: green, max: red.
            {
                "type": "2_color_scale",
                "min_color": "#00FF7F",
                "max_color": "#C82538",
            },
        )

        # Special formatting for all percent columns
        # Note: we can't just search for `percent`, because that's included in the word `percentile`.
        if "percent " in column or "(percent)" in column or "Percent " in column:
            # Make these columns percentages.
            percentage_format = workbook.add_format({"num_format": "0%"})
            worksheet.set_column(
                f"{column_character}:{column_character}",
                column_width,
                percentage_format,
            )

    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "bottom"}
    )

    # Overwrite both the value and the format of each header cell
    # This is because xlsxwriter / pandas has a known bug where it can't wrap text for a dataframe.
    # See https://stackoverflow.com/questions/42562977/xlsxwriter-text-wrap-not-working.
    for col_num, value in enumerate(census_tracts_score_comparison_df.columns.values):
        worksheet.write(0, col_num, value, header_format)

    writer.save()


def compare_census_tracts_scores(
    df: pd.DataFrame,
    index_a: Index,
    index_b: Index,
    output_dir: pathlib.PosixPath,
    comparison_fields: typing.List[str],
):
    # Secondary comparison DF
    census_tracts_score_comparison_df = get_census_tracts_score_comparison_df(
        df=df,
        method_a_priority_census_tracts_field=index_a.priority_communities_field,
        method_b_priority_census_tracts_field=index_b.priority_communities_field,
        comparison_fields=comparison_fields,
    )

    # Write secondary comparison to CSV.
    file_name_part = f"Census tracts comparison output - {index_a.method_name} and {index_b.method_name}"
    output_dir.mkdir(parents=True, exist_ok=True)
    file_path = output_dir / (file_name_part + ".csv")
    file_path_xlsx = output_dir / (file_name_part + ".xlsx")

    census_tracts_score_comparison_df.to_csv(
        path_or_buf=file_path,
        na_rep="",
        index=False,
    )

    write_census_tracts_score_comparison_excel(
        census_tracts_score_comparison_df=census_tracts_score_comparison_df,
        file_path=file_path_xlsx,
    )


for (index_a, index_b) in itertools.combinations(census_tract_indices, 2):
    print(f"Comparing {index_a} and {index_b}.")
    compare_census_tracts_scores(
        df=merged_df,
        index_a=index_a,
        index_b=index_b,
        comparison_fields=comparison_fields,
        output_dir=COMPARISON_OUTPUTS_DIR / "census_tracts_score_comparisons",
    )

Comparing Index(method_name='Definition M', priority_communities_field='Definition M (communities)') and Index(method_name='Definition L', priority_communities_field='Definition L (communities)').
Comparing Index(method_name='Definition M', priority_communities_field='Definition M (communities)') and Index(method_name='Climate Factor (Definition M)', priority_communities_field='Climate Factor (Definition M)').
Comparing Index(method_name='Definition M', priority_communities_field='Definition M (communities)') and Index(method_name='Energy Factor (Definition M)', priority_communities_field='Energy Factor (Definition M)').
Comparing Index(method_name='Definition M', priority_communities_field='Definition M (communities)') and Index(method_name='Transportation Factor (Definition M)', priority_communities_field='Transportation Factor (Definition M)').
Comparing Index(method_name='Definition M', priority_communities_field='Definition M (communities)') and Index(method_name='Housing Factor (

In [22]:
def write_markdown_and_docx_content(
    markdown_content: str,
    file_dir: pathlib.PosixPath,
    file_name_without_extension: str,
) -> pathlib.PosixPath:
    """Write Markdown content to both .md and .docx files."""
    # Set the file paths for both files.
    markdown_file_path = file_dir / f"{file_name_without_extension}.md"
    docx_file_path = file_dir / f"{file_name_without_extension}.docx"

    # Write the markdown content to file.
    with open(markdown_file_path, "w") as text_file:
        text_file.write(markdown_content)

    # Convert markdown file to Word doc.
    pypandoc.convert_file(
        source_file=str(markdown_file_path),
        to="docx",
        outputfile=str(docx_file_path),
        extra_args=[],
    )

    return docx_file_path


def get_markdown_comparing_census_tract_indices(
    census_tract_indices=typing.List[Index],
    df=pd.DataFrame,
    state_field=GEOID_STATE_FIELD_NAME,
) -> str:
    """Generate a Markdown string of analysis of multiple census tract indices."""
    count_field_name = "Count of census tracts"

    # Create markdown content for comparisons.
    markdown_content = f"""
# Comparing multiple indices at the census tract level

(This report was calculated on {datetime.today().strftime('%Y-%m-%d')}.)

This report compares the following indices: {", ".join([index.method_name for index in census_tract_indices])}.

"""

    for (index1, index2) in itertools.combinations(census_tract_indices, 2):
        # First, find out geographic overlap in indices by finding all state and territory
        # names where both indices are not null.
        df_subset_for_states = df[
            [
                state_field,
                index1.priority_communities_field,
                index2.priority_communities_field,
            ]
        ]

        df_subset_for_states = df_subset_for_states.dropna()

        # List of all states/territories in their FIPS codes:
        # TODO: move "This report analyzes the following US states and territories" inside the comparison?
        state_ids = sorted(df_subset_for_states[state_field].unique())
        this_comparison_state_names = ", ".join(
            [us.states.lookup(state_id).name for state_id in state_ids]
        )

        # Group all data by their different values on Priority Communities Field for Index1 vs Priority Communities Field for Index2.
        count_df = (
            df.groupby(
                [
                    index1.priority_communities_field,
                    index2.priority_communities_field,
                ]
            )[ExtractTransformLoad.GEOID_TRACT_FIELD_NAME]
            .count()
            .reset_index(name=count_field_name)
        )

        total_census_tracts = count_df[count_field_name].sum()

        # Returns a series
        true_true_census_tracts_series = count_df.loc[
            count_df[index1.priority_communities_field]
            & count_df[index2.priority_communities_field],
            count_field_name,
        ]
        true_false_census_tracts_series = count_df.loc[
            count_df[index1.priority_communities_field]
            & ~count_df[index2.priority_communities_field],
            count_field_name,
        ]
        false_true_census_tracts_series = count_df.loc[
            ~count_df[index1.priority_communities_field]
            & count_df[index2.priority_communities_field],
            count_field_name,
        ]
        false_false_census_tracts_series = count_df.loc[
            ~count_df[index1.priority_communities_field]
            & ~count_df[index2.priority_communities_field],
            count_field_name,
        ]

        # Convert from series to a scalar value, including accounting for if no data exists for that pairing.
        true_true_census_tracts = (
            true_true_census_tracts_series.iloc[0]
            if len(true_true_census_tracts_series) > 0
            else 0
        )
        true_false_census_tracts = (
            true_false_census_tracts_series.iloc[0]
            if len(true_false_census_tracts_series) > 0
            else 0
        )
        false_true_census_tracts = (
            false_true_census_tracts_series.iloc[0]
            if len(false_true_census_tracts_series) > 0
            else 0
        )
        false_false_census_tracts = (
            false_false_census_tracts_series.iloc[0]
            if len(false_false_census_tracts_series) > 0
            else 0
        )

        markdown_content += (
            "*** \n\n"
            "There are "
            f"{true_true_census_tracts} ({true_true_census_tracts / total_census_tracts:.0%}) "
            f"census tracts that are both {index1.method_name} priority communities and {index2.method_name} priority communities.\n\n"
            "There are "
            f"{true_false_census_tracts} ({true_false_census_tracts / total_census_tracts:.0%}) "
            f"census tracts that are {index1.method_name} priority communities but not {index2.method_name} priority communities.\n\n"
            "There are "
            f"{false_true_census_tracts} ({false_true_census_tracts / total_census_tracts:.0%}) "
            f"census tracts that are not {index1.method_name} priority communities but are {index2.method_name} priority communities.\n\n"
            "There are "
            f"{false_false_census_tracts} ({false_false_census_tracts / total_census_tracts:.0%}) "
            f"census tracts that are neither {index1.method_name} priority communities nor {index2.method_name} priority communities.\n\n"
            f"This comparison analyzed the following US states and territories: {this_comparison_state_names}.\n\n"
            "\n\n"
        )

    return markdown_content


def get_comparison_census_tract_indices(
    census_tract_indices=typing.List[Index],
    df=pd.DataFrame,
    state_field=GEOID_STATE_FIELD_NAME,
) -> pathlib.PosixPath:
    markdown_content = get_markdown_comparing_census_tract_indices(
        census_tract_indices=census_tract_indices,
        df=df,
    )

    comparison_docx_file_path = write_markdown_and_docx_content(
        markdown_content=markdown_content,
        file_dir=COMPARISON_OUTPUTS_DIR,
        file_name_without_extension=f"Comparison report - All census tract indices",
    )

    return comparison_docx_file_path


# Compare multiple scores at the census tract level
get_comparison_census_tract_indices(
    census_tract_indices=census_tract_indices,
    df=merged_with_state_information_df,
)

/var/folders/z6/9czv8cpx2hvcycd6slvfp4ph0000gq/T/ipykernel_35872/2594933895.py:125: RuntimeWarning: invalid value encountered in long_scalars
  f"{true_true_census_tracts} ({true_true_census_tracts / total_census_tracts:.0%}) "
/var/folders/z6/9czv8cpx2hvcycd6slvfp4ph0000gq/T/ipykernel_35872/2594933895.py:128: RuntimeWarning: invalid value encountered in long_scalars
  f"{true_false_census_tracts} ({true_false_census_tracts / total_census_tracts:.0%}) "
/var/folders/z6/9czv8cpx2hvcycd6slvfp4ph0000gq/T/ipykernel_35872/2594933895.py:131: RuntimeWarning: invalid value encountered in long_scalars
  f"{false_true_census_tracts} ({false_true_census_tracts / total_census_tracts:.0%}) "
/var/folders/z6/9czv8cpx2hvcycd6slvfp4ph0000gq/T/ipykernel_35872/2594933895.py:134: RuntimeWarning: invalid value encountered in long_scalars
  f"{false_false_census_tracts} ({false_false_census_tracts / total_census_tracts:.0%}) "


PosixPath('/Users/emmausds/Desktop/justice40-tool/data/data-pipeline/data_pipeline/data/comparison_outputs/20220217-115053/Comparison report - All census tract indices.docx')

In [23]:
# Note: this is helpful because this file is long-running, so it alerts the user when the
# data analysis is done. Can be removed when converted into scripts. -LMB.
import os

os.system("say 'data analysis is written.'")

0